# Bronze Layer - Ingestão de Dados Brutos# Objetivo: Carregar os dados brutos do Censo 2022 para a camada Bronze# Dados: População por sexo, cor/raça, instrução, crescimento, domicílio, território, rendimento

In [ ]:
import sysimport ossys.path.append(os.path.abspath('../src'))import pandas as pdimport numpy as npfrom pyspark.sql import SparkSessionfrom pyspark.sql import functions as Ffrom sqlalchemy import create_engine, textimport warningswarnings.filterwarnings('ignore')print("Bibliotecas importadas ")

Bibliotecas importadas

In [ ]:
# Sparkspark = SparkSession.builder \.appName("Bronze_Layer_Censo_2022") \.config("spark.driver.memory", "4g") \.getOrCreate()print(f"Spark {spark.version} iniciado")

/usr/local/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not foundSetting default log level to "WARN".To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).25/10/17 21:00:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable

Spark 3.5.1

In [ ]:
# PostgreSQLPOSTGRES_HOST = os.getenv('POSTGRES_HOST', 'localhost')POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')POSTGRES_DB = os.getenv('POSTGRES_DB', 'etl_censo')POSTGRES_USER = os.getenv('POSTGRES_USER', 'etl_user')POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'etl_password')postgres_url = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"engine = create_engine(postgres_url, pool_pre_ping=True)with engine.connect() as conn:result = conn.execute(text("SELECT version();"))print("Conexão PostgreSQL estabelecida")

PostgreSQL conectado

In [ ]:
DATA_DIR = "../data"csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]print(f"{len(csv_files)} arquivos CSV encontrados")

17 arquivos CSV encontrados

# 1. População por Sexo

In [5]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - População por sexo - Brasil.csv")df = df.select(F.col("Sexo").alias("sexo"),F.col("População(pessoas)").cast("long").alias("populacao_pessoas"),F.col("Recorte Geográfico").alias("recorte_geografico")).withColumn("ano", F.lit(2022))df.toPandas().to_sql('populacao_sexo', engine, schema='bronze', if_exists='replace', index=False)print(f" bronze.populacao_sexo: {df.count()} registros")

bronze.populacao_sexo: 2 registros

# 2. População por Cor/Raça

In [6]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - População por cor ou raça - Brasil.csv")df = df.select(F.col("Cor ou raça").alias("cor_ou_raca"),F.col("População (pessoas)").cast("long").alias("populacao_pessoas"),F.col("Recorte Geográfico").alias("recorte_geografico")).withColumn("ano", F.lit(2022))df.toPandas().to_sql('populacao_cor_raca', engine, schema='bronze', if_exists='replace', index=False)print(f" bronze.populacao_cor_raca: {df.count()} registros")

bronze.populacao_cor_raca: 5 registros

# 3. Nível de Instrução

In [7]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - Nível de instrução - Brasil.csv")df = df.select(F.col("Nível de instrução").alias("nivel_instrucao"),F.col("População (pessoas)").cast("long").alias("populacao_pessoas"),F.col("Recorte Geográfico").alias("recorte_geografico")).withColumn("ano", F.lit(2022))df.toPandas().to_sql('nivel_instrucao', engine, schema='bronze', if_exists='replace', index=False)print(f" bronze.nivel_instrucao: {df.count()} registros")

bronze.nivel_instrucao: 4 registros

# 4. Crescimento Populacional

In [8]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - Crescimento Populacional - Brasil.csv")df = df.select(F.col("Ano da pesquisa").cast("int").alias("ano_pesquisa"),F.col("População(pessoas)").cast("long").alias("populacao_pessoas"),F.col("Recorte Geográfico").alias("recorte_geografico"))df.toPandas().to_sql('crescimento_populacional', engine, schema='bronze', if_exists='replace', index=False)print(f" bronze.crescimento_populacional: {df.count()} registros")

bronze.crescimento_populacional: 13 registros

# 5. Situação do Domicílio

In [9]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - População por situação do domicílio - Brasil.csv")df = df.select(F.col("Situação").alias("situacao"),F.col("População (pessoas)").cast("long").alias("populacao_pessoas"),F.col("Percentual").cast("decimal(5,2)").alias("percentual"),F.col("Recorte Geográfico").alias("recorte_geografico")).withColumn("ano", F.lit(2022))df.toPandas().to_sql('populacao_situacao_domicilio', engine, schema='bronze', if_exists='replace', index=False)print(f" bronze.populacao_situacao_domicilio: {df.count()} registros")

bronze.populacao_situacao_domicilio: 2 registros

# 6. Características dos Domicílios

In [10]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - Características dos domicílios - Brasil.csv")df = df.select(F.col("Característica").alias("caracteristica"),F.col("Não possui(%)").cast("decimal(5,2)").alias("nao_possui_percentual"),F.col("Possui(%)").cast("decimal(5,2)").alias("possui_percentual"),F.col("Recorte Geográfico").alias("recorte_geografico")).withColumn("ano", F.lit(2022))df.toPandas().to_sql('caracteristicas_domicilios', engine, schema='bronze', if_exists='replace', index=False)print(f" bronze.caracteristicas_domicilios: {df.count()} registros")

bronze.caracteristicas_domicilios: 4 registros

# 7. Território

In [11]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - Território - Brasil.csv")df = df.select(F.col("Ano da pesquisa").cast("int").alias("ano_pesquisa"),F.col("Área(km²)").cast("decimal(15,2)").alias("area_km2"),F.col("Densidade demográfica(hab/km²)").cast("decimal(10,2)").alias("densidade_demografica"),F.col("Recorte Geográfico").alias("recorte_geografico"))df.toPandas().to_sql('territorio', engine, schema='bronze', if_exists='replace', index=False)print(f" bronze.territorio: {df.count()} registros")

bronze.territorio: 1 registros

# 8. Pirâmide Etária (população 14+ anos)

In [12]:
df = spark.read.option("header", "true").option("sep", ";").csv(f"{DATA_DIR}/Censo 2022 - Pirâmide etária - Brasil.csv")pop_14_mais = df.filter(~F.col("Grupo de idade").isin(["0 a 4 anos", "5 a 9 anos", "10 a 14 anos"]))total_14_mais = pop_14_mais.select((F.col("População feminina(pessoas)").cast("long") + F.col("População masculina(pessoas)").cast("long")).alias("pop")).agg(F.sum("pop")).collect()[0][0]print(f" População 14+ anos: {total_14_mais:,}")

População 14+ anos: 162,951,495

# 9. Dados Sintéticos de Rendimento# Importante: Os dados do IBGE não contêm informações de rendimento/salário# Vamos gerar dados sintéticos realistas baseados na distribuição conhecida do Brasil

In [13]:
# UFs brasileiras com regiõesufs_regioes = [('AC','N','Acre'),('AM','N','Amazonas'),('AP','N','Amapá'),('PA','N','Pará'),('RO','N','Rondônia'),('RR','N','Roraima'),('TO','N','Tocantins'),('AL','NE','Alagoas'),('BA','NE','Bahia'),('CE','NE','Ceará'),('MA','NE','Maranhão'),('PB','NE','Paraíba'),('PE','NE','Pernambuco'),('PI','NE','Piauí'),('RN','NE','Rio Grande do Norte'),('SE','NE','Sergipe'),('ES','SE','Espírito Santo'),('MG','SE','Minas Gerais'),('RJ','SE','Rio de Janeiro'),('SP','SE','São Paulo'),('PR','S','Paraná'),('RS','S','Rio Grande do Sul'),('SC','S','Santa Catarina'),('DF','CO','Distrito Federal'),('GO','CO','Goiás'),('MS','CO','Mato Grosso do Sul'),('MT','CO','Mato Grosso')]print(f"Total UFs: {len(ufs_regioes)}")

Total UFs: 27

In [14]:
SM_2022 = 1302.00POP_TOTAL = 203080756POP_OCUPADA = int(POP_TOTAL * 0.60)# Faixas de rendimento: (nome, sm_min, sm_max, % população) - Baseado em dados reais IBGEfaixas_sm = [('ate_1sm', 0, 1, 0.37), # 37% até 1 SM('1_a_2sm', 1, 2, 0.25), # 25% de 1 a 2 SM('2_a_5sm', 2, 5, 0.18), # 18% de 2 a 5 SM('5_a_20sm', 5, 20, 0.15), # 15% de 5 a 20 SM('mais_20sm', 20, 50, 0.05) # 5% mais de 20 SM]print(f"População ocupada estimada: {POP_OCUPADA:,}")print(f"Salário mínimo 2022: R$ {SM_2022:.2f}")

População ocupada estimada: 121,848,453Salário mínimo 2022: R$ 1302.00

In [15]:
np.random.seed(42)dados = []for uf, regiao, nome_uf in ufs_regioes:# População ocupada por UF (distribuição aproximada)if uf == 'SP': pop_uf = int(POP_OCUPADA * 0.22)elif uf in ['MG','RJ']: pop_uf = int(POP_OCUPADA * 0.10)elif uf in ['BA','RS','PR']: pop_uf = int(POP_OCUPADA * 0.06)else: pop_uf = int(POP_OCUPADA * 0.02)for faixa, sm_min, sm_max, prop in faixas_sm:pop_faixa = int(pop_uf * prop)rend_medio = ((sm_min + sm_max) / 2) * SM_2022# Variação regional (Sudeste/Sul ganham mais)if regiao in ['SE','S']: rend_medio *= 1.2elif regiao in ['N','NE']: rend_medio *= 0.8# Distribuir por sexo (55% M, 45% F)for sexo, prop_sexo in [('M', 0.55), ('F', 0.45)]:pop_sexo = int(pop_faixa * prop_sexo)rend_sexo = rend_medio * (0.8 if sexo == 'F' else 1.0) # Mulheres ganham 20% menos# Distribuir por raça/corfor raca, prop_raca in [('B',0.43),('P',0.45),('N',0.10),('A',0.01),('I',0.01)]:pop_raca = int(pop_sexo * prop_raca)rend_final = rend_sexo * (0.75 if raca in ['N','P'] else 1.0) # Negros/pardos ganham 25% menosif pop_raca > 0:dados.append({'uf': uf,'cod_municipio': '0000000','sexo': sexo,'raca_cor': raca,'faixa_rendimento_sm': faixa,'rendimento_mensal_nominal': round(rend_final, 2),'populacao_ocupada': pop_raca,'ano': 2022})df_rend = pd.DataFrame(dados)print(f" Dados sintéticos gerados: {len(df_rend):,} registros")print(f"População ocupada total: {df_rend['populacao_ocupada'].sum():,}")

Dados sintéticos gerados: 1,350 registrosPopulação ocupada total: 124,284,490

In [16]:
# Validação dos dados sintéticosprint("\n Validação DOS DADOS SINTÉTICOS")print("="*60)# Distribuição por faixaprint("\n1. Distribuição por Faixa de Rendimento:")dist = df_rend.groupby('faixa_rendimento_sm')['populacao_ocupada'].sum()total = dist.sum()for f, p in dist.items():print(f" {f:15s}: {100*p/total:5.2f}%")# Rendimento por regiãodf_rend['regiao'] = df_rend['uf'].map({u: r for u, r, _ in ufs_regioes})rend_regiao = df_rend.groupby('regiao').apply(lambda x: (x['rendimento_mensal_nominal'] * x['populacao_ocupada']).sum() / x['populacao_ocupada'].sum())print("\n2. Rendimento Médio por Região:")for r, v in rend_regiao.items():print(f" {r:5s}: R$ {v:,.2f}")# Rendimento médio Brasilrend_brasil = (df_rend['rendimento_mensal_nominal'] * df_rend['populacao_ocupada']).sum() / df_rend['populacao_ocupada'].sum()print(f"\n3. Rendimento Médio Brasil: R$ {rend_brasil:,.2f}")# Por sexorend_sexo = df_rend.groupby('sexo').apply(lambda x: (x['rendimento_mensal_nominal'] * x['populacao_ocupada']).sum() / x['populacao_ocupada'].sum())print("\n4. Rendimento Médio por Sexo:")for s, v in rend_sexo.items():sexo_nome = 'Masculino' if s == 'M' else 'Feminino'print(f" {sexo_nome}: R$ {v:,.2f}")

Validação DOS DADOS SINTÉTICOS============================================================1. Distribuição por Faixa de Rendimento:1_a_2sm : 25.00%2_a_5sm : 18.00%5_a_20sm : 15.00%ate_1sm : 37.00%mais_20sm : 5.00%2. Rendimento Médio por Região:CO : R$ 4,920.42N : R$ 3,936.33NE : R$ 3,936.34S : R$ 5,904.54SE : R$ 5,904.563. Rendimento Médio Brasil: R$ 5,132.714. Rendimento Médio por Sexo:Feminino: R$ 4,512.26Masculino: R$ 5,640.34

In [17]:
# Salvar dados de rendimentodf_rend.drop('regiao', axis=1).to_sql('censo_trabalhadores', engine, schema='bronze', if_exists='replace', index=False)print("\n bronze.censo_trabalhadores criada ")

bronze.censo_trabalhadores criada

# Resumo Bronze Layer

In [18]:
with engine.connect() as conn:result = conn.execute(text("""SELECT table_name FROM information_schema.tablesWHERE table_schema = 'bronze' ORDER BY table_name"""))print("\n" + "="*70)print(" Resumo da camada BRONZE")print("="*70)print(f"{'Tabela':<40} {'Registros':>15}")print("-"*70)total_reg = 0for row in result:count_result = conn.execute(text(f"SELECT COUNT(*) FROM bronze.{row[0]}"))count = count_result.fetchone()[0]print(f"{row[0]:<40} {count:>15,}")total_reg += countprint("-"*70)print(f"{'TOTAL':<40} {total_reg:>15,}")print("="*70)

======================================================================Resumo da camada BRONZE======================================================================Tabela Registros----------------------------------------------------------------------caracteristicas_domicilios 4censo_trabalhadores 1,350crescimento_populacional 13nivel_instrucao 4populacao_cor_raca 5populacao_sexo 2populacao_situacao_domicilio 2territorio 1----------------------------------------------------------------------TOTAL 1,381======================================================================

In [19]:
print("\n" + "="*70)print(" BRONZE LAYER CONCLUÍDA concluída")print("="*70)print("\n Dados carregados:")print(" População por sexo")print(" População por cor/raça")print(" Nível de instrução")print(" Crescimento populacional histórico")print(" Situação do domicílio (urbano/rural)")print(" Características dos domicílios")print(" Território e densidade")print(" Dados sintéticos de rendimento (27 UFs)")print("\n Próximo passo: Executar silver.ipynb para limpeza e normalização")

======================================================================BRONZE LAYER CONCLUÍDA concluída======================================================================Dados carregados:População por sexoPopulação por cor/raçaNível de instruçãoCrescimento populacional históricoSituação do domicílio (urbano/rural)Características dos domicíliosTerritório e densidadeDados sintéticos de rendimento (27 UFs)Próximo passo: Executar silver.ipynb para limpeza e normalização

In [20]:
spark.stop()print(" Spark Session encerrada")

Spark Session encerrada